# 🔄 DataSens E1 — Notebook 4 : Tests CRUD Complets

**🎯 Objectif** : Démontrer les opérations CRUD (Create, Read, Update, Delete) sur les tables principales

---

## 📋 Contenu de ce notebook

1. **CRUD "C" (Create)** : Insertion de documents, météo, indicateurs
2. **CRUD "R" (Read)** : Requêtes jointes complexes
3. **CRUD "U" (Update)** : Mise à jour de champs
4. **CRUD "D" (Delete)** : Suppression contrôlée (ON DELETE)
5. **Contrôles qualité** : Détection doublons, %NULL par colonne
6. **KPIs** : Counts par source/type_donnee, par thème/événement

---

## 🔒 RGPD & Gouvernance

⚠️ **Rappel** : Suppressions avec ON DELETE CASCADE pour intégrité référentielle



In [ ]:
# Configuration (réutiliser depuis notebooks précédents)
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "notebooks" else NOTEBOOK_DIR
load_dotenv(PROJECT_ROOT / ".env")

PG_HOST = os.getenv("POSTGRES_HOST", "localhost")
PG_PORT = int(os.getenv("POSTGRES_PORT", "5432"))
PG_DB = os.getenv("POSTGRES_DB", "datasens")
PG_USER = os.getenv("POSTGRES_USER", "ds_user")
PG_PASS = os.getenv("POSTGRES_PASS", "ds_pass")

PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(PG_URL, future=True)

print("✅ Connexion PostgreSQL établie")
print(f"   📍 {PG_HOST}:{PG_PORT}/{PG_DB}")


## ✅ CRUD "C" (CREATE) : Insertion de données

Insertion d'exemples pour tester les contraintes d'intégrité


In [ ]:
print("📝 CRUD CREATE - Insertion d'exemples")
print("=" * 80)

with engine.begin() as conn:
    # 1. Créer un document
    result = conn.execute(text("""
        INSERT INTO document (titre, texte, langue, hash_fingerprint)
        VALUES (:titre, :texte, :langue, :hash)
        RETURNING id_doc
    """), {
        "titre": "Test CRUD Create",
        "texte": "Document de test pour démonstration CRUD",
        "langue": "fr",
        "hash": "test_hash_1234567890abcdef"
    })
    id_doc = result.scalar()
    print(f"✅ Document créé : id_doc = {id_doc}")

    # 2. Créer un relevé météo
    # D'abord s'assurer qu'un territoire existe
    result = conn.execute(text("""
        INSERT INTO territoire (ville, code_insee, lat, lon)
        VALUES ('Paris', '75056', 48.8566, 2.3522)
        ON CONFLICT (code_insee) DO UPDATE SET code_insee = EXCLUDED.code_insee
        RETURNING id_territoire
    """))
    id_territoire = result.scalar()

    result = conn.execute(text("""
        INSERT INTO meteo (id_territoire, date_obs, temperature, humidite, vent_kmh, pression, meteo_type)
        VALUES (:t, NOW(), :temp, :hum, :vent, :pres, :type)
        RETURNING id_meteo
    """), {
        "t": id_territoire,
        "temp": 18.5,
        "hum": 65.0,
        "vent": 15.0,
        "pres": 1013.25,
        "type": "CLOUDS"
    })
    id_meteo = result.scalar()
    print(f"✅ Relevé météo créé : id_meteo = {id_meteo}")

    # 3. Créer un indicateur
    result = conn.execute(text("""
        SELECT id_type_indic FROM type_indicateur WHERE code = 'POPULATION'
    """)).scalar()

    if result:
        id_type_indic = result
        conn.execute(text("""
            INSERT INTO indicateur (id_territoire, id_type_indic, valeur, annee)
            VALUES (:t, :ti, :val, :annee)
        """), {
            "t": id_territoire,
            "ti": id_type_indic,
            "val": 2161000.0,
            "annee": 2023
        })
        print("✅ Indicateur créé pour Paris (population 2023)")

print("\n✅ CRUD CREATE terminé !")


## 📖 CRUD "R" (READ) : Requêtes jointes

Lecture des données avec jointures complexes


In [ ]:
print("📖 CRUD READ - Requêtes jointes")
print("=" * 80)

# Requête 1 : Documents avec territoire et source
query1 = """
SELECT
    d.id_doc,
    LEFT(d.titre, 50) as titre_extrait,
    d.langue,
    t.ville,
    s.nom as source,
    f.date_collecte
FROM document d
LEFT JOIN territoire t ON d.id_territoire = t.id_territoire
LEFT JOIN flux f ON d.id_flux = f.id_flux
LEFT JOIN source s ON f.id_source = s.id_source
ORDER BY d.id_doc DESC
LIMIT 10;
"""

df_read = pd.read_sql(query1, engine)
print(f"\n📄 {len(df_read)} documents avec jointures :\n")
print(df_read.to_string(index=False))

# Requête 2 : Météo avec territoire
query2 = """
SELECT
    t.ville,
    m.date_obs,
    m.temperature,
    m.humidite,
    m.meteo_type
FROM meteo m
JOIN territoire t ON m.id_territoire = t.id_territoire
ORDER BY m.date_obs DESC
LIMIT 5;
"""

df_meteo = pd.read_sql(query2, engine)
print("\n🌦️ Derniers relevés météo :\n")
print(df_meteo.to_string(index=False))

print("\n✅ CRUD READ terminé !")


## ✏️ CRUD "U" (UPDATE) : Mise à jour

Modification de champs existants


In [ ]:
print("✏️ CRUD UPDATE - Mise à jour")
print("=" * 80)

with engine.begin() as conn:
    # Mettre à jour un document
    result = conn.execute(text("""
        UPDATE document
        SET langue = :langue, titre = :titre
        WHERE id_doc = (
            SELECT id_doc FROM document
            WHERE titre LIKE '%CRUD%'
            LIMIT 1
        )
        RETURNING id_doc, titre, langue
    """), {
        "langue": "fr",
        "titre": "Test CRUD Update - Modifié"
    })

    row = result.fetchone()
    if row:
        print(f"✅ Document mis à jour : id_doc={row[0]}, titre='{row[1]}', langue='{row[2]}'")
    else:
        print("⚠️ Aucun document à mettre à jour")

print("\n✅ CRUD UPDATE terminé !")


## 🗑️ CRUD "D" (DELETE) : Suppression contrôlée

Suppression avec vérification des contraintes ON DELETE


In [ ]:
print("🗑️ CRUD DELETE - Suppression contrôlée")
print("=" * 80)

with engine.begin() as conn:
    # Compter avant suppression
    count_before = conn.execute(text("SELECT COUNT(*) FROM document WHERE titre LIKE '%CRUD%'")).scalar()
    print(f"📊 Documents 'CRUD' avant suppression : {count_before}")

    # Supprimer un document (ON DELETE SET NULL pour id_flux)
    conn.execute(text("""
        DELETE FROM document
        WHERE titre LIKE '%CRUD%' AND id_doc IN (
            SELECT id_doc FROM document
            WHERE titre LIKE '%CRUD%'
            LIMIT 1
        )
    """))

    count_after = conn.execute(text("SELECT COUNT(*) FROM document WHERE titre LIKE '%CRUD%'")).scalar()
    print(f"📊 Documents 'CRUD' après suppression : {count_after}")
    print(f"   ✅ {count_before - count_after} document(s) supprimé(s)")

print("\n✅ CRUD DELETE terminé !")


## 🔍 Contrôles qualité

Détection des doublons et vérification des valeurs NULL


In [ ]:
print("🔍 Contrôles qualité")
print("=" * 80)

with engine.connect() as conn:
    # Doublons fingerprint
    dup_query = """
    SELECT hash_fingerprint, COUNT(*) as c
    FROM document
    WHERE hash_fingerprint IS NOT NULL
    GROUP BY hash_fingerprint
    HAVING COUNT(*) > 1;
    """
    df_dup = pd.read_sql(dup_query, conn)
    print(f"\n🔎 Doublons fingerprint : {len(df_dup)}")
    if len(df_dup) > 0:
        print(df_dup.head())
    else:
        print("   ✅ Aucun doublon détecté")

    # %NULL par colonne
    null_query = """
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN titre IS NULL THEN 1 ELSE 0 END)::float / COUNT(*) * 100 as pct_null_titre,
        SUM(CASE WHEN texte IS NULL THEN 1 ELSE 0 END)::float / COUNT(*) * 100 as pct_null_texte,
        SUM(CASE WHEN langue IS NULL THEN 1 ELSE 0 END)::float / COUNT(*) * 100 as pct_null_langue
    FROM document;
    """
    df_null = pd.read_sql(null_query, conn)
    print("\n📊 Pourcentage NULL par colonne :")
    print(df_null.to_string(index=False))

print("\n✅ Contrôles qualité terminés !")


## 📊 KPIs : Statistiques par source/type/thème

Comptages et agrégations pour visualisation


In [ ]:
print("📊 KPIs - Statistiques")
print("=" * 80)

with engine.connect() as conn:
    # KPI 1 : Counts par type_donnee
    kpi1 = """
    SELECT
        td.libelle as type_source,
        COUNT(DISTINCT d.id_doc) as nb_documents,
        COUNT(DISTINCT s.id_source) as nb_sources
    FROM document d
    LEFT JOIN flux f ON d.id_flux = f.id_flux
    LEFT JOIN source s ON f.id_source = s.id_source
    LEFT JOIN type_donnee td ON s.id_type_donnee = td.id_type_donnee
    GROUP BY td.libelle
    ORDER BY nb_documents DESC;
    """
    df_kpi1 = pd.read_sql(kpi1, conn)
    print("\n📦 Documents par type de source :")
    print(df_kpi1.to_string(index=False))

    # KPI 2 : Counts par thème
    kpi2 = """
    SELECT
        t.libelle as theme,
        COUNT(DISTINCT e.id_event) as nb_evenements,
        COUNT(DISTINCT de.id_doc) as nb_documents_associes
    FROM theme t
    LEFT JOIN evenement e ON t.id_theme = e.id_theme
    LEFT JOIN document_evenement de ON e.id_event = de.id_event
    GROUP BY t.libelle
    ORDER BY nb_evenements DESC;
    """
    df_kpi2 = pd.read_sql(kpi2, conn)
    print("\n🏷️ Événements par thème :")
    print(df_kpi2.to_string(index=False))

print("\n✅ CRUD complet testé avec succès !")
print("   ➡️ Passez au notebook 05_snapshot_and_readme.ipynb")
